In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer
import xgboost as xgb
from matplotlib import pyplot as plt

import sys
sys.path.append('..')
from utils import preprocess, missing, evaluate

OSError: [WinError 126] The specified module could not be found

### Globals and load data

In [2]:
target = 'SI.POV.DDAY'
predict_year=2010
#percent of input Indicators to use (set to 100 for full set of input features)
percent = 50

#Load the data from disk
input_dir = '.\\..\\data\\'
data_input = "cleaned_data.pkl"
data = pd.read_pickle(input_dir + data_input)

#Possible subset of data choosen to reduce calulation time
#For percetages less than 100% we try to choose a subset that represents the spread of variables

if percent == 100:
    pass
else: 
    num_indicators_original = data.shape[1]
    step = int(100/percent)
    data_new = data.iloc[:,::step].copy()
    #Add the target column if not already included
    if target not in data_new.columns:
        data_new[target] = data[target]
    data = data_new
    
print(data.shape[1], "indicators included")

307 indicators included


In [3]:

# jagNote
# save pk. data back to a csv (for use in Azure)

import pickle as pkl
# import pandas as pd

input_dir2 = '.\\..\\data\\'
data_input2 = "cleaned_data.pkl"

with open(input_dir2 + data_input2, "rb") as f2:
    object = pkl.load(f2)
    
df2 = pd.DataFrame(object)
df2.to_csv(input_dir2 + r'cleaned_data2.csv')





### Window data and preprocess

In [4]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target, impute_type='interpolation')

#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

Wall time: 2min 35s


In [5]:
print(data_train_regressors)

Indicator Code            NY.ADJ.NNTY.KD.ZG                      \
lag                                       1         2         3   
country            window                                         
Argentina          2                21.2166   12.3954   5.75809   
                   3                7.48641   21.2166   12.3954   
                   4               -1.00319   7.48641   21.2166   
                   5                11.3624  -1.00319   7.48641   
                   6               -11.4711   11.3624  -1.00319   
                   7               -4.92054  -11.4711   11.3624   
                   8              -0.985731  -4.92054  -11.4711   
                   9               -5.67288 -0.985731  -4.92054   
                   10                3.0834  -5.67288 -0.985731   
Armenia            2                14.6338   16.8789   9.44762   
                   3                14.8584   14.6338   16.8789   
                   4                10.3311   14.8584   14.633

In [6]:
X_train = data_train_regressors_subset.values
y_train = data_train_targets_subset.values.ravel()
X_test = data_test_regressors_subset.values
y_test = data_test_targets_subset

In [7]:
print(type(data_train_regressors_subset))
print(data_train_regressors_subset.shape)
print(data_train_targets_subset.shape)
print(data_test_regressors_subset.shape)
print(data_test_targets_subset.shape)

# 
# combine the dataframes vertically
xxx = pd.concat([data_train_regressors_subset,data_test_regressors_subset])
print(xxx.shape)

yyy = pd.concat([data_train_targets_subset,data_test_targets_subset])
print(yyy.shape)

# combine horizontally (to add target back to a combined dataframe)
xxxyyy = pd.concat([xxx, yyy.set_index(xxx.index)], axis=1)

# write out to a csv file

input_dir5 = '.\\..\\data\\'
    
xxxyyy.to_csv(input_dir5 + r'combined_data5.csv')



<class 'pandas.core.frame.DataFrame'>
(455, 921)
(455, 1)
(81, 921)
(81, 1)
(536, 921)
(536, 1)


In [8]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

# print(X_train.size)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

#
# combine train and test arrrays
#
X_combined = np.vstack((X_train, X_test))
print(X_combined.shape)
y_train3 = y_train.reshape((455,1))
y_combined = np.vstack((y_train3, y_test))
print(y_combined.shape)

xy_combined = np.concatenate((X_combined, y_combined),axis=1)


input_dir3 = '.\\..\\data\\'

#
# write numpy array back to csv



np.savetxt(input_dir3 + r'combined_data3.csv', xy_combined, delimiter=",")



<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
(455, 921)
(455,)
(81, 921)
(81, 1)
(536, 921)
(536, 1)


### XGBoost Model

#### Out-of-the-box using the Scikit-learn interface

In [8]:
XGB = xgb.XGBRegressor(random_state=42 ,objective='reg:squarederror', subsample=0.9)
XGB.fit( X_train,y_train)
#Make predictions
predictions = XGB.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of XGBoost out-of-the-box is:", np.sqrt(mse))

RMSE of XGBoost out-of-the-box is: 5.062785174026983


#### Tuning of the Algorithm

In [9]:
cv_folds = 5

scorer = make_scorer(mean_squared_error ,greater_is_better=False)

Step 1. Tune the number of estimators

In [10]:
model = xgb.XGBRegressor(random_state=42,
                         objective='reg:squarederror',
                         max_depth=5, 
                         min_child_weight = 1, 
                         gamma = 0, 
                         subsample=0.9, 
                         colsample_bytree = 0.8, 
                         scale_pos_weight = 1)

param = model.get_xgb_params()
data_matrix = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(param, data_matrix, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=50)
#Set the optimised number of estimators
model.set_params(n_estimators=cvresult.shape[0])
print("Optimal number of estimators:", cvresult.shape[0])

Optimal number of estimators: 100


In [8]:
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 1) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 1) is: 5.418787503109928


Step 2. Tune max_depth and min_child_weight

In [11]:
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 2) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 2) is: 5.252244194544083


The result of our test shows that the performance of the model after tuning was actually worse than before. The model is generalising very poorly. This may be a reflection on some kind of bias introduced in creating our training and data subsets. It would be worth looking at how I decided to discard any countries early on that did not have target values for the target year, 2010. It may have made more sense to window the data and split into training and test subsets and then, after, discard any observations that did not have a target value. 

In [12]:
grid_model.best_params_

{'max_depth': 9, 'min_child_weight': 1}

Step 3. Tune Gamma

In [13]:
params = {
 'gamma':[i/10.0 for i in range(0,5)]
}



grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 3) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 3) is: 5.252244194544083


In [14]:
grid_model.best_params_

{'gamma': 0.0}

Step 4. Tune Regularization

In [15]:
params = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1]
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 4) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 4) is: 5.139286222900817


In [16]:
grid_model.best_params_

{'reg_alpha': 1e-05}

In [17]:
params = {
 'reg_lambda':[1e-5, 1e-2, 0.1, 1]
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 5) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 5) is: 5.139286222900817


### XGBoost Model with no Imputation

In [18]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target)

#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

X_train_miss = data_train_regressors_subset.values
y_train_miss = data_train_targets_subset.values.ravel()
X_test_miss = data_test_regressors_subset.values
y_test_miss = data_test_targets_subset


Wall time: 2.11 s


#### Out-of-the-box xgboost on data without imputation

Hand-tuning of the XGBoost model. I pick some important paramneters and play around until I get a good result. I'm sure there is more accuracy that can be obtained from this model by gridsearching but I think this is enough to illustrate that using a XGBoost (or perhaps any tree-based predictive algo) without any imputation done on the input data gives by far the best results of any of the models tried.

In [19]:
XGB = xgb.XGBRegressor(n_estimators=200,  objective='reg:squarederror',max_depth=7, subsample=0.87, reg_lambda=0.2)
XGB.fit( X_train_miss,y_train_miss)
#Make predictions
predictions = XGB.predict(X_test_miss) 

mse= mean_squared_error(y_test_miss, predictions)
print("RMSE of XGBoost out-of-the-box is:", np.sqrt(mse))

RMSE of XGBoost out-of-the-box is: 4.9376482439849445
